### Text-generation with pre-trained RLHF Hugging Face models 

In [33]:
# !pip install transformers datasets
!pip install scikit-learn

  Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl (8.0 MB)
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   ------------ --------------------------- 11.0/36.3 MB 54.3 MB/s eta 0:00:01
   ----------------------- ---------------- 21.8/36.3 MB 54.1 MB/s eta 0:00:01
   ---------------------------- ----------- 25.7/36.3 MB 41.1 MB/s eta 0:00:01
   -------------------------------------- - 34.6/36.3 MB 41.1 MB/s eta 0:00:01
   ---------------------------------------- 36.3/36.3 MB 39.1 MB/s  0:00:00
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
model_name = "lvwerra/gpt2-imdb-pos-v2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
text_generator = pipeline("text-generation",model=model,tokenizer=tokenizer)
review_prompt = "It felt like a great start for the movie, but"
generated_review = text_generator(review_prompt, max_length=20, truncation=True)
print("Generated review:",generated_review)

c:\Users\cheku\GenAI-RoadMap\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cheku\.cache\huggingface\hub\models--lvwerra--gpt2-imdb-pos-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP d

Generated review: [{'generated_text': 'It felt like a great start for the movie, but I was very impressed by how much music and artwork we saw. We had a lot of fun with the whole movie. It was a very fun movie to see it. It was a very fun movie to see the movie. It was a fun movie to see the movie because of many of the people who have recorded it. It was a very fun movie to see the movie because of many people who have the freedom to have it. It was a very fun movie to see all the people who have made it, and it was a lot fun to see the movie with the music and the music. It was a very fun movie to see the movie with the story and the movie. It was a very fun movie to see the movie with the music and the movies. It was a very fun movie to see the movie with the movie and the movie. It was a very fun movie to see the movie to see the movie because of many of the people who have made it. It was a very fun movie to see the movie because of many of the people who have made it. It was a ve

In [10]:
generated_review = text_generator("The start was a great one, but the end disappointed me.", max_new_tokens=8, max_length=10, truncation=True)
print(generated_review)

Both `max_new_tokens` (=8) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'The start was a great one, but the end disappointed me. was very good. The story was very'}]


In [17]:
generated_review = text_generator("Suprisingly, the film is a", max_new_tokens=3, max_length=10, truncation=True)
print(generated_review)

Both `max_new_tokens` (=3) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Suprisingly, the film is a great masterpiece.'}]


### Classifying the sentiment of the generated review

In [12]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
sentiment_analyzer = pipeline('sentiment-analysis',model=DistilBertForSequenceClassification.from_pretrained('lvwerra/distilbert-imdb'),tokenizer=DistilBertTokenizerFast.from_pretrained('lvwerra/distilbert-imdb'))


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


In [18]:
review_text = "Suprisingly, the film is a very good one" # One of the generations
sentiment = sentiment_analyzer(review_text)
print(sentiment)

[{'label': 'POSITIVE', 'score': 0.9941137433052063}]


### Fine-tuning a pre-trained model

In [20]:
from datasets import load_dataset

dataset = load_dataset("argilla/tripadvisor-hotel-reviews")
tokenizer = AutoTokenizer.from_pretrained('openai-gpt')

tokenizer.add_special_tokens({'pad_token':'[PAD]'})

def tokenize_function(examples):
    return tokenizer(examples['text'],padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 20491/20491 [00:03<00:00, 6455.80 examples/s]


In [21]:
help(tokenized_datasets)

Help on DatasetDict in module datasets.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
 |
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __enter__(self)
 |
 |  __exit__(self, exc_type, exc_val, exc_tb)
 |
 |  __getitem__(self, k) -> datasets.arrow_dataset.Dataset
 |      Return self[key].
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  align_labels_with_mapping(self, label2id: dict, label_column: str) -> 'DatasetDict'
 |      Align the dataset's label ID and label name mapping to match an input `label2id` mapping.
 |      This is useful when you want to ensure that a model's predicted labels are aligned with the dataset.
 |      The alignment in done using the lowercase label names.
 |
 |      Args:
 |          label2id (`dict`):
 |              The label name to ID mapping to align the da

In [23]:
tokenized_datasets['train']

Dataset({
    features: ['text', 'inputs', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'multi_label', 'explanation', 'id', 'metadata', 'status', 'event_timestamp', 'metrics', 'input_ids', 'attention_mask'],
    num_rows: 20491
})

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    do_eval=True,
    do_predict=False,
    do_train=False,
    eval_accumulation_steps=None,
    eval_delay=0,
    eval_do_concat_batches=True,
    eval_on_start=False,
    eval_steps=None,
    eval_strategy='epoch',
    eval_use_gather_object=False,
    evaluation_strategy='epoch',
    fp16=False,
    fp16_backend='auto',
    fp16_full_eval=False,
    fp16_opt_level='O1',
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs=None,
    hub_strategy='every_save',
    learning_rate=5e-05,
    logging_dir='output_dir/runs/Jan19_07-20-19_0b6b572e-c52f-4580-96c7-1f579483693e',
    logging_steps=500,
    logging_strategy='steps',
    lr_scheduler_kwargs={},
    lr_scheduler_type='linear',
    max_grad_norm=1.0,
    max_steps=-1,
    neftune_noise_alpha=None,
    no_cuda=False,
    num_train_epochs=3.0,
    optim='adamw_torch',
    optim_args=None,
    optim_target_modules=None,
    output_dir='output_dir',
    overwrite_output_dir=False,
    past_index=-1,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    prediction_loss_only=False,
    push_to_hub=False,
    push_to_hub_model_id=None,
    push_to_hub_organization=None,
    save_on_each_node=False,
    save_only_model=False,
    save_safetensors=True,
    save_steps=500,
    save_strategy='steps',
    save_total_limit=None,
    seed=42,
    skip_memory_metrics=True,
    split_batches=None,
    tf32=None,
    warmup_ratio=0.0,
    warmup_steps=0,
    weight_decay=0.0,
)

# Define the train and test datasets
training_dataset = tokenized_datasets['train']
testing_dataset = tokenized_datasets['test']

# Initialize the trainer class
trainer = Trainer(
# Add arguments to the class
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset
)

### Loading the Prompt and Preference Datasets

In [27]:
preference_dataset = load_dataset('trl-internal-testing/hh-rlhf-helpful-base-trl-style',split='train')

def extract_prompt(text):
    return text[0]['content']

preference_dataset_with_prompt = preference_dataset.map(lambda sample: {**sample, 'prompt': extract_prompt(sample['chosen'])})

sample = preference_dataset_with_prompt.select(range(1))
print(sample['prompt'])

Column(['Hi, I want to learn to play horseshoes. Can you teach me?'])


### Detecting Anomalous Reviews

In [30]:
import numpy as np

prob_dists = np.array([np.array([[0.53463835, 0.46536162]]),
 np.array([[0.6142045 , 0.38579544]]),
 np.array([[0.5121281 , 0.48787192]])])

texts = ['This movie was fantastic!', "I don't know if I liked it or not.", 'The book was incredibly boring.']

In [31]:
def least_confidence(prob_dist):    
    simple_least_conf = np.nanmax(prob_dist)
    num_labels = float(prob_dist.size)  # number of labels
    least_conf = (1 - simple_least_conf) * (num_labels / (num_labels - 1))
    return least_conf

def filter_low_confidence_predictions(prob_dists, threshold=0.5):
    filtered_indices = [i for i, prob_dist in enumerate(prob_dists) if least_confidence(prob_dist) > threshold]
    return filtered_indices

filtered_indices = filter_low_confidence_predictions(prob_dists)

high_confidence_texts = [texts[i] for i in filtered_indices]
print("High-confidence texts:", high_confidence_texts)


High-confidence texts: ['This movie was fantastic!', "I don't know if I liked it or not.", 'The book was incredibly boring.']


In [34]:
from sklearn.cluster._kmeans import KMeans

def detect_anomalies(data, n_clusters=3):
    # Initialize k-means
    kmeans=KMeans(n_clusters,random_state=42)
    clusters = kmeans.fit_predict(data)
    centers = kmeans.cluster_centers_

    # Calculate distances from cluster centers
    distances = np.linalg.norm(data-centers[clusters],axis=1)
    return distances
  
confidences = np.array([[0.34],
       [0.72],
       [0.51],
       [0.68]])

anomalies = detect_anomalies(confidences)
print(anomalies)

[0.   0.02 0.   0.02]


### Active Learning

In [37]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to c:\users\cheku\appdata\local\temp\pip-req-build-qltecfs0
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Created wheel for modAL-python: filename=modal_python-0.4.2-py3-none-any.whl size=32999 sha256=dd4f7e0e9f86826b6d3cb977e27fc0a23f62b22561579264739dfbaea5b4ec98
  Stored in directory: C:\Users\cheku\AppData\Local\Temp\pip-ephem-wheel-cache-acq7j_wa\wheels\73\44\60\526028e94d205df2a0991c99476d1298559184803e4d6d774a
Successfully built m

  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git 'C:\Users\cheku\AppData\Local\Temp\pip-req-build-qltecfs0'


In [ ]:
from modAL.models.learners import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.linear_model import LogisticRegression

X_labeled = np.array([[ 0.57985729,  1.14796125,  1.21897079, ..., -0.12951725,
         1.00561139, -0.05658849],
       [-0.21048803, -0.83714743, -0.46264474, ..., -0.98991531,
        -0.67658159,  1.23661974],
       [ 0.86018521,  0.48524158,  0.96373788, ...,  1.39659785,
         0.36755707,  1.11643075],
       ...,
       [-1.44311529, -1.39557951, -0.87960631, ..., -2.23979163,
        -1.20240545, -0.99144228],
       [ 0.94686146, -1.73188868,  0.91339109, ...,  0.06429488,
        -1.46065958,  1.35108895],
       [ 1.18705115,  0.83332305, -0.28231204, ...,  0.31912084,
         0.64395896, -0.79062399]])

y_labeled=np.array([0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1])

# Create the active learner object
learner = ActiveLearner(
    # Set the estimator 
    estimator=LogisticRegression(),
    # Set the query strategy
    query_strategy = uncertainty_sampling,
    # Pass the labeled data
    X_training=X_labeled, y_training=y_labeled
)

In [ ]:
n_queries=10
for _ in range(n_queries):
    # Use the current labeled data
    learner.teach(X_labeled,y_labeled)
    # Query from unlabeled data
    query_idx, _ = learner.query(X_unlabeled,n_instances=5)
    X_new, y_new = X_unlabeled[query_idx], y[query_idx]  
    X_labeled = np.vstack((X_labeled, X_new))  
    y_labeled = np.append(y_labeled, y_new)  
    # Update the unlabeled dataset
    X_unlabeled = np.delete(X_unlabeled, query_idx, axis=0)